In [132]:
%run cgibbs.ipynb

In [133]:
import pyreadr
import pandas as pd
import numpy as np

In [134]:
# load in data

sim = pyreadr.read_r('~/sim.RData')
Psi0 = pyreadr.read_r('~/hyp_Psi0.RData')
mu0 = pyreadr.read_r('~/hyp_mu0.RData')
alpha = pyreadr.read_r('~/hyp_alpha.RData')
kappa0 = pyreadr.read_r('~/hyp_kappa0.RData')
retained_classes = pd.read_csv("~/retained_classes.txt", sep="\t", header=None)

In [182]:
# assign run_mcmc() parameters

Psi0 = Psi0['Psi0'].values
mu0 = mu0['mu0'].values
alpha = alpha['alpha'].values.T[0]
kappa0 = kappa0['kappa0'].values.T[0]

dat = sim['sim.dat']
dm = dat.shape[1]
n = dat.shape[0]
nm = len(alpha)
nw = 1

alph = alpha
hyp = [kappa0, mu0, Psi0]
reduced_classes = retained_classes
nstep = 1000
tune_df = np.repeat(1000., nm)
reduced_classes = retained_classes
param = []
for i in range(nw) :
    dict_list = []
    for m in range(nm) :
        dictionary = {"mu":[],"Sigma":[]}
        Sigma = np.array(np.matrix(Psi0[:,:,m]).getH())
        mu = mu0[m]
        dictionary['mu']= mu
        dictionary['Sigma']= Sigma
        dict_list.append(dictionary)
    z = np.random.choice(np.arange(1, nm+1), size=n, p=np.array(alph)/np.sum(alph), replace=True)
    chain_i = [dict_list, alph, z]
    param.append(chain_i)
labels = np.apply_along_axis(lambda x: ''.join(map(str, x + 1)), axis=1, arr=retained_classes)

In [183]:
# run an MCMC

result_chain, acpt_rate_chain, tune_df_chain = run_mcmc(dat, param, hyp, alpha, nstep, labels, tune_df, opt_rate=0.3)

In [189]:
# mu_chain output

mu_chains = []
for i in range(nm):
    mu_chains_i = []
    for j in range(nstep+1):
        mu_chains_i.append(result_chain[j][0][0][i]['mu'].tolist())
    mu_chains.append(mu_chains_i)
    
mu_chains_reshaped = np.array(mu_chains).reshape(np.array(mu_chains).shape[0], -1)
np.savetxt('mu_chains_reshaped.txt', mu_chains_reshaped)

In [191]:
# Sigma_chain output

Sigma_chains = []
for i in range(nm):
    Sigma_chains_i = []
    for j in range(nstep+1):
        Sigma_chains_i.append(result_chain[j][0][0][i]['Sigma'].tolist())
    Sigma_chains.append(Sigma_chains_i)

Sigma_chains_reshaped = np.array(Sigma_chains).reshape(np.array(Sigma_chains).shape[0], -1)
np.savetxt('Sigma_chains_reshaped.txt', Sigma_chains_reshaped)

In [249]:
# prop_chain output

prop_chain = []
for j in range(nstep+1):
    prop_chain.append(result_chain[j][0][1])

np.savetxt('prop_chain.txt', prop_chain)

In [211]:
# z_chain output

z_chain = []
for j in range(nstep+1):
    z_chain.append(result_chain[j][0][2])

np.savetxt('z_chain.txt', z_chain)